# <span style="color:#0080FF;">Udacity - Data Analyst Nanodegree </span>
## <span style="color:#B45F04;">Project 3: Wrangle OpenStreetMap Data</span>
### <span style="color:black;">Created by: Layne Newhouse</span>
### <span style="color:#6E6E6E;">Submitted: October 4, 2016</span>

### *Background Information*
For the OpenStreetMap project I chose to look at the Toronto Ontario Canada area, where I currently live. The following url is a link to the 79MB, uncompressed, OSM XML file for the Toronto area which is provided by MapZen.

https://mapzen.com/data/metro-extracts/metro/toronto_canada/

The bulk of this project will be based on programmatically extracting and cleaning the OpenStreetMap (OSM) data around Toronto, exploring that data in order to develop a deeper understanding of what we have and what types of problems might arise, and then exporting the cleaned data into an SQL database. Once the data is in the SQL database the following questions from the rubric will be answered:

- What is the size of the file?
- What are the total number of nodes and ways in this dataset?
- How many unique users have contributed to this OSM dataset?
- Extract the number of nodes found for particular node types i.e. cafe, shop etc. 
- Any additional questions or statistics that might come up after exploring the dataset.

After these questions have been answered I will discuss further ideas for improvements on the dataset as well as benefits and problems that could result from these improvements.

### Exploring the extracted OSM file

I begin the data wrangling process by parsing through the tags in the extracted OSM XML file. Each unique tag encountered will be added to the dictionary 'tags' with the tag name being the key and the value starting at '1'. For each addition tag encountered with the same key, the value will be increased by one until each tag has been looked at and accounted for.

In [1]:
import xml.etree.cElementTree as ET
from pprint import pprint

OSMFILE = 'toronto_canada.osm'
tag_tags = []

def count_tags(filename):
        tags = {}
        tag = []
        for event, elem in ET.iterparse(filename):
            tag.append(elem.tag)
        for i in tag:
            if i in tags:
                tags[i] += 1
            else:
                tags[i] = 1         
        return tags

def test():
    tags = count_tags(OSMFILE)
    pprint(tags)
    tag_tags.append(tags['tag'])
   
if __name__ == "__main__":
    test()

{'bounds': 1,
 'member': 96337,
 'nd': 5542308,
 'node': 4873350,
 'osm': 1,
 'relation': 6429,
 'tag': 4820840,
 'way': 710944}


The output dictionary 'tags' as seen above highlights the different tags found in the OSM file. Nothing from this dictionary seems out of the ordinary so we will continue forward with a general idea of the number of node and way tags that we will be dealing with. The list 'tag_tags' was added and will be used in a later section of the code in order to get a better understanding of the tags tagged <'tag'>.

Next we will explore the keys associated with each 'tag' tag in order to assess the types of keys that we will be working with. We will begin by defining the different types of regular expressions that we will be looking for i.e. all lower cases, lower cases with one colon in between, problem character etc. The code will then parse through each element and look for elements tagged 'tag', then extract the 'k' value, or the key, from these tags and add them to the appropriate bin based on the re.search expression. Within this section of code, there are many commented out print statements that I chose to leave in the code because I believe that during the data exploration phase, there is no 'one code fits all' and it is important to always be altering the code to achieve a greater understanding of the dataset that you are working with.

In [3]:
import xml.etree.cElementTree as ET
from pprint import pprint
import re
import operator

OSMFILE = 'toronto_canada.osm'
lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
lower_2colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')
omit_keys = []

def key_type(element, keys, other, problem_characters):
    if element.tag == "tag":
        k = element.get("k")
        if re.search(lower,k):
            keys['lower'] += 1
        elif re.search(lower_colon,k):
            keys['lower_colon'] += 1
        elif re.search(lower_2colon,k):
            keys['lower_2colon'] += 1
        elif re.search(problemchars,k):
            keys['problemchars'] += 1
            if k in problem_characters:
                problem_characters[k] += 1
            else:
                problem_characters[k] = 1
        else:
            keys['other'] += 1
            if k in other:
                other[k] += 1
            else:
                other[k] = 1
    return keys

def process_map(filename):
    keys = {"lower": 0, "lower_colon": 0, "lower_2colon": 0, "problemchars": 0, "other": 0}
    other = {}
    problem_characters = {}
    for _, element in ET.iterparse(filename):
        keys = key_type(element, keys, other, problem_characters)
    sort_other = sorted(other.items(), key=operator.itemgetter(1), reverse=True)
    sort_problem_characters = sorted(problem_characters.items(), 
                                     key=operator.itemgetter(1), reverse=True)
    #The two print statements below will display problem tags and problem characters
    #as well as the amount of times that these tag appears
    #print "OTHER:", sort_other
    #print "OTHER:", sort_problem_characters
    
    gct_count = 0
    for key in other:
        omit_keys.append(key)
        if 'geobase' in key or 'canvec' in key or 'tiger' in key:
            #print key, other[key]
            gct_count += other[key]
    for key in problem_characters:
        omit_keys.append(key)
    unique_not_gct = 0 
    for i in omit_keys:
        if 'geobase' not in i and 'canvec' not in i and 'tiger' not in i:
            unique_not_gct += 1
            #print i
    
    print                                                                                
    print "Problem tags make up %f%% of the total %d tags in the 'Toronto Canada' dataset" % (100*float(keys['other'])/tag_tags[0], 
                                                                                               tag_tags[0])
    print "%d out of the %d problem tags (%d%%) come from geobase, canvec and tiger tags" % (gct_count, keys['other'], 
                                                                                             100*float(gct_count)/keys['other'])
    print "i.e. 'geobase:acquisitionTechnique', 'canvec:UUID', 'tiger:name_base_1'"
    print "The remaining %d (%f%%) tags can either be changed by hand or ignored;" % (keys['other']-gct_count, 
                                                                                    100*(keys['other']-gct_count)/float(tag_tags[0]))
    print"of the remaining problem tags %d are unique." % unique_not_gct
    print
    #The print statment below will show all tags that are being omitted from this dataset
    #print omit_keys
    return keys

def test():
    keys = process_map(OSMFILE)
    pprint(keys)

if __name__ == "__main__":
    test()


Problem tags make up 2.508214% of the total 4820840 tags in the 'Toronto Canada' dataset
115230 out of the 120917 problem tags (95%) come from geobase, canvec and tiger tags
i.e. 'geobase:acquisitionTechnique', 'canvec:UUID', 'tiger:name_base_1'
The remaining 5687 (0.117967%) tags can either be changed by hand or ignored;
of the remaining problem tags 142 are unique.

{'lower': 2769359,
 'lower_2colon': 5398,
 'lower_colon': 1924345,
 'other': 120917,
 'problemchars': 821}


The output dictionary, as can be seen above, highlights the different compositions of tags within this database. The vast majority of tag keys fall within one of the first three types (lower case with either zero, one, or two colons) which we are ready to deal with and are excepting to see in the following data cleaning sections. 'problemchars' are keys with characters that have been noted as problematic (i.e. [=\+/&<>;\'"\?%#$@\,\. \t\r\n]). For the purpose of the project, these keys will be ignored but will be addressed later in this document as an area of further improvement for the dataset. The 'other' dictionary key is filled with keys that do not fall into any of the other categories, in order to get a better understanding of these keys were I used the commented out 'sort_other' list to view all of the 'other' tags sorted by frequency of occurrence. When doing this, I found that the majority of these keys came from geobase, canvec, and tiger which seem to be names of programs that have assisted in the filling out of elements in the openStreetMaps datasets. When doing a search in a sample raw xml dataset, I found that the nodes and ways that were tagged with these types of tags always had other tags that would compliment the node or way. Thus I decided that these tags were of no value to my dataset and would be removed for clarity and cleaning purposes. 

Finally I have used a few print statements to provide me with some statistics regarding the problematic tags. From here I found out that these 'other' tag keys make up approximately 2.5 percent of the total tag keys but when I remove the geobase, canvec and tiger tags from this count it drops down significantly to approximately 0.1 percent or 142 unique keys. For the sake of this project, these keys will be ignored but diccussed later on in the improvements discussion section. 

### Cleaning the Data

The following functions marks the beginning of the data cleaning phase. This function specifically parses through elements looking for tags specifically with key ('k') values of "addr:street" and then analyses the value ('v') associated with that tag. Once the street address 'value' is obtained, the function extracts the last word, which in most cases will be the street type, and compares these values to the excepted street types.

In [4]:
import xml.etree.cElementTree as ET
import re
from pprint import pprint
from collections import defaultdict

OSMFILE = "toronto_canada.osm"
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)
updated_street_names = {}

# referenced: https://en.wikipedia.org/wiki/Street_suffix for common street names
expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
            "Trail", "Parkway", "Commons", "Crescent", "Line", "Circle", "Gardens", "Close", "Concession",
            "Gate", "Grove", "Heights", "Hill", "Path", "Run", "Sideroad", "Terrace", "Way", "Townline",
            "North", "East", "South", "West"]

# 'Highway  5  West' => 'Highway 5 West' 
# 'Sideroad 5 Tosorontio' => 'Tosorontio Sideroad 5'
mapping = { "St": "Street",
            "St.": "Street",
            "street": "Street",
            "STREET": "Street",
            "Ave": "Avenue",
            "Ave.": "Avenue",
            "avenue": "Avenue",
            "Rd": "Road",
            "Rd.": "Road",
            "road": "Road",
            "Blvd": "Boulevard",
            "Blvd.": "Boulevard",
            "Fernway": "Fern Way",
            "By-pass": "Bypass",
            "Cir": "Circle",
            "Cres": "Crescent",
            "Cresent": "Crescent",
            #"Crest": "Crescent", -> Individual cases...
            "Crt": "Court",
            "Ct": "Court",
            "Cv": "Court",
            "Dr": "Drive",
            "Driver": "Drive",
            "E": "East",
            "E.": "East",
            "Hrbr": "Harbour Way",
            "Lan": "Lane",
            "Lanes": "Lane",
            "Ldg": "Landing",
            "N": "North",
            "S": "South",
            "S.": "South",
            "Trl": "Trail",
            "W": "West",
            "W.": "West",
            }

def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)

def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")

def audit(osmfile):
    osm_file = open(osmfile, "r")
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
    osm_file.close()
    return street_types

def update_name(name, mapping):
    #re.sub(pattern, repl, string, count=0, flags=0)
    m = street_type_re.search(name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            if street_type in mapping:
                name = re.sub(street_type, mapping[street_type], name)
    return name

def test():
    st_types = audit(OSMFILE)
    print "The dataset contains %d street type tags" % len(st_types)
    #pprint(dict(st_types))
    count = 0
    for st_type, ways in st_types.iteritems():
        for name in ways:
            better_name = update_name(name, mapping)
            if better_name != name:
                # The print statment below shows all street names that were altered
                #print name, "=>", better_name
                count += 1
                updated_street_names[name] = better_name
    print "of these, %d tags were updated for constistancy" % count
    # the variable updated_street_names contains a dictionary using the old names as the key 
    # and the updated names as the values
    #print updated_street_names

if __name__ == '__main__':
    test()

The dataset contains 389 street type tags
of these, 95 tags were updated for constistancy


When the function above was run for the first few times, the street names that did not show up as expected were printed out in a list. At this point, some manual labour had to be done in order to fill in the 'mapping' dictionary as can be seen in the body of the function above. This was done by observing the unexpected street types which were printed out and then adding each respective mapping to the 'mapping' dictionary so that the next time the function was run, the street names would be updated and removed from the unexpected list. The commented out '#print name, "=>", better_name' prints out all of the updated street types as they are found in the update_street_types dictionary. This dictionary will be used in a subsequent function in order to replace old street names with the updated street names before being added to the SQL database. As an additional note, there were some cases where suffixes such as 'Crest' could not be properly mapped due to the fact that some instances of the word were referring to just 'Crest' where others were referring to the word 'Crescent'. Cleaning of this type would have to be done in more detail if desired. 

### Cleaning cont. & Exporting to .csv

The code below pulls in the Toronto Canada OSM data, parses through it, cleans some of the data based on the dictionaries and lists passed on from the previous functions, shapes the data based on the schema provided, and exports the cleaned data to .csv files which are ready to be imported into the SQL database. The create_tag_dictionary function is where all of the previous work in the data cleaning process is realized. For each element, the tag key is analyzed and checked against problem characters, the omit_keys list, and the updated_street_names dictionary. If any key shows up in one of these checks, the key and/or value is either changed appropriately or omitted. 

In [5]:
import csv
import codecs
import re
import xml.etree.cElementTree as ET
import cerberus
import schema

OSM_PATH = "toronto_canada.osm"
NODES_PATH = "nodes.csv"
NODE_TAGS_PATH = "node_tags.csv"
WAYS_PATH = "ways.csv"
WAY_NODES_PATH = "way_nodes.csv"
WAY_TAGS_PATH = "way_tags.csv"

LOWER_COLON = re.compile(r'^([a-z]|_)+:([a-z]|_)+')
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

SCHEMA = schema.schema

# Make sure the fields order in the csvs matches the column order in the sql table schema
NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']

# ================================================== #
#               Cleaning & Shaping                   #
# ================================================== #

def shape_element(element, node_attr_fields=NODE_FIELDS, way_attr_fields=WAY_FIELDS,
                  problem_chars=PROBLEMCHARS, default_tag_type='regular'):
    """Clean and shape node or way XML element to Python dict"""

    node_attribs = {}
    way_attribs = {}
    way_nodes = []
    tags = []  # Handle secondary tags the same way for both node and way elements

    if element.tag == 'node':
        for field in NODE_FIELDS:
            node_attribs[field] = element.attrib[field]
        for tag in element.iter("tag"):    
            tags = create_tag_dict(element, tag, tags, default_tag_type)
        #print {'node': node_attribs, 'node_tags': tags} ...debugging
        return {'node': node_attribs, 'node_tags': tags}
    
    elif element.tag == 'way':
        for field in WAY_FIELDS:
            way_attribs[field] = element.attrib[field]
        
        position = 0
        for node in element.iter("nd"):
            node_dict = {'id': element.attrib['id'], 'node_id': node.attrib['ref'], 'position': position}
            position += 1
            way_nodes.append(node_dict)
            
        for tag in element.iter("tag"):    
            tags = create_tag_dict(element, tag, tags, default_tag_type)
        #print {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}    
        return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}

def create_tag_dict(element, tag, tags, tag_type):
    k = tag.attrib['k']
    if not re.search(PROBLEMCHARS, k):
        # The following three lines omit problem keys found earlier as well as update
        # old street names with the improved street names from the previous script
        if k not in omit_keys:
            if k in updated_street_names:
                k = updated_street_names[k]
            tag_dict = {}
            tag_dict['id'] = element.attrib['id']
            if ":" not in k:
                tag_dict['key'] = k
            else:
                tag_type = tag.attrib['k'].split(':',1)[0]
                k = tag.attrib['k'].split(':',1)[1]
                tag_dict['key'] = k
            tag_dict['value'] = tag.attrib['v']
            tag_dict['type'] = tag_type
            tags.append(tag_dict)
    return tags

        

# ================================================== #
#               Helper Functions                     #
# ================================================== #
def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag"""

    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


def validate_element(element, validator, schema=SCHEMA):
    """Raise ValidationError if element does not match schema"""
    if validator.validate(element, schema) is not True:
        field, errors = next(validator.errors.iteritems())
        message_string = "\nElement of type '{0}' has the following errors:\n{1}"
        error_strings = (
            "{0}: {1}".format(k, v if isinstance(v, str) else ", ".join(v))
            for k, v in errors.iteritems()
        )
        raise cerberus.ValidationError(
            message_string.format(field, "\n".join(error_strings))
        )


class UnicodeDictWriter(csv.DictWriter, object):
    """Extend csv.DictWriter to handle Unicode input"""

    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
            k: (v.encode('utf-8') if isinstance(v, unicode) else v) for k, v in row.iteritems()
        })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)


# ================================================== #
#               Main Function                        #
# ================================================== #
def process_map(file_in, validate):
    """Iteratively process each XML element and write to csv(s)"""

    with codecs.open(NODES_PATH, 'w') as nodes_file, \
         codecs.open(NODE_TAGS_PATH, 'w') as node_tags_file, \
         codecs.open(WAYS_PATH, 'w') as ways_file, \
         codecs.open(WAY_NODES_PATH, 'w') as way_nodes_file, \
         codecs.open(WAY_TAGS_PATH, 'w') as way_tags_file:

        nodes_writer = UnicodeDictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = UnicodeDictWriter(node_tags_file, NODE_TAGS_FIELDS)
        ways_writer = UnicodeDictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = UnicodeDictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = UnicodeDictWriter(way_tags_file, WAY_TAGS_FIELDS)

        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()

        validator = cerberus.Validator()

        for element in get_element(file_in, tags=('node', 'way')):
            el = shape_element(element)
            if el:
                if validate is True:
                    validate_element(el, validator)

                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                elif element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])


if __name__ == '__main__':
    # Note: Validation is ~ 10X slower. For the project consider using a small
    # sample of the map when validating.
    process_map(OSM_PATH, validate=False)


### Setting up the SQL Database

The procedure of setting up the SQL Database in python is fairly systematic. It starts by connecting sqlite3 to the toronto_canada database filename, creating a cursor object and then executing commands to create each of the individual tables. After the schemas are added and the changes are committed, the program goes row by row through the csv files pulling them into python and then inserting them into the new SQL database by table and row.

In [6]:
import sqlite3
import csv
from pprint import pprint

nodes_csv = 'nodes.csv'
node_tags_csv = 'node_tags.csv'
ways_csv = 'ways.csv'
way_tags_csv = 'way_tags.csv'
way_nodes_csv = 'way_nodes.csv'

# Connect to the database, if it doesn't exist it will be created
sqlite_file = 'toronto_canada.db'
db = sqlite3.connect(sqlite_file)
db.text_factory = str
# Create a cursor object
c = db.cursor()

# Create the tables
c.execute('''
CREATE TABLE nodes (
    id INTEGER PRIMARY KEY NOT NULL,
    lat REAL,
    lon REAL,
    user TEXT,
    uid INTEGER,
    version INTEGER,
    changeset INTEGER,
    timestamp TEXT)
''')
c.execute('''
CREATE TABLE node_tags (
    id INTEGER,
    key TEXT,
    value TEXT,
    type TEXT,
    FOREIGN KEY (id) REFERENCES nodes(id))
''')
c.execute('''
CREATE TABLE ways (
    id INTEGER PRIMARY KEY NOT NULL,
    user TEXT,
    uid INTEGER,
    version TEXT,
    changeset INTEGER,
    timestamp TEXT)
''')
c.execute('''
CREATE TABLE way_tags (
    id INTEGER NOT NULL,
    key TEXT NOT NULL,
    value TEXT NOT NULL,
    type TEXT,
    FOREIGN KEY (id) REFERENCES ways(id))
''')
c.execute('''
CREATE TABLE way_nodes (
    id INTEGER NOT NULL,
    node_id INTEGER NOT NULL,
    position INTEGER NOT NULL,
    FOREIGN KEY (id) REFERENCES ways(id),
    FOREIGN KEY (node_id) REFERENCES nodes(id))    
''')
# Commit changes
db.commit()

# Read in the data
with open(nodes_csv, 'rb') as read_csv:
    reader = csv.DictReader(read_csv) #comma is default delimiter
    nodes_db = [(i['id'], i['lat'], i['lon'], i['user'], i['uid'], i['version'], i['changeset'], i['timestamp']) for i in reader]
with open(node_tags_csv, 'rb') as read_csv:
    reader = csv.DictReader(read_csv) #comma is default delimiter
    node_tags_db = [(i['id'], i['key'], i['value'], i['type']) for i in reader]
with open(ways_csv, 'rb') as read_csv:
    reader = csv.DictReader(read_csv) #comma is default delimiter
    ways_db = [(i['id'], i['user'], i['uid'], i['version'], i['changeset'], i['timestamp']) for i in reader]
with open(way_tags_csv, 'rb') as read_csv:
    reader = csv.DictReader(read_csv) #comma is default delimiter
    way_tags_db = [(i['id'], i['key'], i['value'], i['type']) for i in reader]
with open(way_nodes_csv, 'rb') as read_csv:
    reader = csv.DictReader(read_csv) #comma is default delimiter
    way_nodes_db = [(i['id'], i['node_id'], i['position']) for i in reader]
    
# Insert formatted data
c.executemany('''
    INSERT INTO nodes(id, lat, lon, user, uid, version, changeset, timestamp) 
    VALUES (?, ?, ?, ?, ?, ?, ?, ?);''', nodes_db)
c.executemany('''
    INSERT INTO node_tags(id, key, value, type) 
    VALUES (?, ?, ?, ?);''', node_tags_db)
c.executemany('''
    INSERT INTO ways(id, user, uid, version, changeset, timestamp) 
    VALUES (?, ?, ?, ?, ?, ?);''', ways_db)
c.executemany('''
    INSERT INTO way_tags(id, key, value, type) 
    VALUES (?, ?, ?, ?);''', way_tags_db)
c.executemany('''
    INSERT INTO way_nodes(id, node_id, position) 
    VALUES (?, ?, ?);''', way_nodes_db)
db.commit()
db.close()

### Queries

The following SQL queries are performed on the cleaned Toronto Canada OpenStreetMaps database (toronto_canada.db) in order to answer the initial questions posed at the beginning of this document as well as further explore the database and come up with additional statistics making use of one or more of the tables within the database.

In [7]:
db = sqlite3.connect(sqlite_file)
c = db.cursor()
c.execute ('PRAGMA PAGE_SIZE')
page_size = c.fetchall()
c.execute ('PRAGMA PAGE_COUNT')
page_count = c.fetchall()
db.close()
# Page size shows the size of the pages in bytes
# Page count shows the amount of pages that are allocated to the database
page_size = page_size[0][0]
page_count = page_count[0][0]
print "The size of this database is %d kB" % (page_size*page_count/1024)

The size of this database is 681844 kB


The above query uses the PRAGMA function to access internal (non-table) data from the database. The total size of the database is the product of the size of each page multiplied by the number of pages in the database.

In [8]:
db = sqlite3.connect(sqlite_file)
c = db.cursor()
c.execute ('''
    SELECT *
    FROM (SELECT count(id) FROM ways
    UNION SELECT count(id) FROM nodes)
''')
idWaysNodes = c.fetchall()
c.execute ('''
    SELECT COUNT(id)
    FROM (SELECT id FROM ways
    UNION SELECT id FROM nodes)
''')
idUnique = c.fetchall()
db.close()
# changing the order in which ways and nodes appear in the union 
# does not seem to change the order of the final table
print "There are %d node tags and %d way tags combining to a total of %d way/node tags." % (idWaysNodes[1][0], idWaysNodes[0][0], 
                                                                                            idWaysNodes[1][0] + idWaysNodes[0][0])
print "Of these tags, %d are not unique between the nodes and the ways tables." % (idWaysNodes[1][0] + idWaysNodes[0][0]
                                                                                   - idUnique[0][0])

There are 4873350 node tags and 710944 way tags combining to a total of 5584294 way/node tags.
Of these tags, 506 are not unique between the nodes and the ways tables.


In [9]:
db = sqlite3.connect(sqlite_file)
c = db.cursor()
c.execute ('''
    SELECT COUNT(DISTINCT uid)
    FROM (SELECT uid FROM nodes
    UNION SELECT uid FROM ways)
''')
all_rows = c.fetchall()
db.close()
unique_uid = all_rows[0][0]
print "%d unique users have contributed to the node and way tags for this database" % unique_uid

1985 unique users have contributed to the node and way tags for this database


In [10]:
db = sqlite3.connect(sqlite_file)
c = db.cursor()
c.execute ('''
    SELECT value, COUNT(*)
    FROM node_tags
    WHERE key = 'amenity'
    GROUP BY value
    ORDER BY COUNT(*) DESC
    LIMIT 10
    ''')
all_rows = c.fetchall()
pprint(all_rows)
db.close()

[(u'fast_food', 2996),
 (u'restaurant', 2744),
 (u'bench', 2352),
 (u'post_box', 1964),
 (u'cafe', 1409),
 (u'parking', 1233),
 (u'waste_basket', 1171),
 (u'bank', 1046),
 (u'fuel', 1001),
 (u'pharmacy', 743)]


The above queries answer the initial questions presented at the beginning of the project document, where:

- The size of this database is 681,844 kB
- The total number of nodes and ways are 4,873,350 and 710,944 respectively
- There are 1985 unique users contributing specifically to this database
- There are 1409 Cafe nodes, 2996 Fast Food nodes and 1046 Bank nodes to name a few

The following queries are a result of further investigation into the dataset.

In [11]:
db = sqlite3.connect(sqlite_file)
c = db.cursor()
c.execute ('''
    SELECT key, COUNT(*)
    FROM node_tags
    GROUP BY key
    ORDER BY COUNT(*) DESC
    LIMIT 10
    ''')
all_rows = c.fetchall()
pprint(all_rows)
db.close()

[(u'source', 483041),
 (u'street', 472140),
 (u'housenumber', 472136),
 (u'city', 414610),
 (u'highway', 85423),
 (u'country', 50682),
 (u'name', 35732),
 (u'state', 26598),
 (u'amenity', 25980),
 (u'province', 23890)]


In [12]:
db = sqlite3.connect(sqlite_file)
c = db.cursor()
c.execute ('''
    SELECT value, COUNT(*)
    FROM node_tags
    WHERE key = 'city'
    GROUP BY value
    ORDER BY COUNT(*) DESC
    LIMIT 20
    ''')
all_rows = c.fetchall()
pprint(all_rows)
db.close()

[(u'City of Toronto', 111651),
 (u'City of Hamilton', 40073),
 (u'Mississauga', 35113),
 (u'City of Brampton', 25557),
 (u'City of Vaughan', 16962),
 (u'Town of Markham', 16852),
 (u'Oakville', 15435),
 (u'City of Oshawa', 12031),
 (u'Richmond Hill', 11346),
 (u'City of Burlington', 11050),
 (u'Town of Whitby', 10638),
 (u'Town of Ajax', 7437),
 (u'Town of Milton', 7397),
 (u'City of Pickering', 6754),
 (u'Town of Caledon', 6701),
 (u'Town of Newmarket', 5851),
 (u'Town of Halton Hills', 5834),
 (u'Toronto', 5121),
 (u'Burlington', 4653),
 (u'Town of Whitchurch-Stouffville', 4398)]


Having first looked at the top ten keys in node_tags, I had then refined my query to do a search on the top twenty 'city' tags within this table to see which cities had the highest number of tags. From here an opportunity to further clean the data became apparent as there were separate tags for 'City of Toronto' and 'Toronto' which could easily be combined. 

In [13]:
db = sqlite3.connect(sqlite_file)
c = db.cursor()
# Shows the top 10 most common keys within way_tags
c.execute ('''
    SELECT key, COUNT(*)
    FROM way_tags
    GROUP BY key
    ORDER BY COUNT(*) DESC
    LIMIT 10
    ''')
all_rows = c.fetchall()
pprint(all_rows)
db.close()

[(u'source', 464856),
 (u'highway', 242805),
 (u'interpolation', 226512),
 (u'surface', 161827),
 (u'lanes', 150670),
 (u'name', 145372),
 (u'building', 102326),
 (u'attribution', 87188),
 (u'access', 59372),
 (u'is_in', 49761)]


In [14]:
db = sqlite3.connect(sqlite_file)
c = db.cursor()
c.execute ('''
    SELECT COUNT(*) FROM 
    (SELECT COUNT(*) as nodeCount
    FROM way_nodes
    GROUP BY id)
    WHERE nodeCount = 1
    ''')
all_rows = c.fetchall()
db.close()
print "%d ways have only a single node assigned to them" % all_rows[0][0]

545 ways have only a single node assigned to them


After trying out many other queries associated with the tag keys and values I also began to investigate timestamps to see what kind of information I could derive.

In [16]:
db = sqlite3.connect(sqlite_file)
c = db.cursor()
# Number of ways and nodes added to the database each year 
c.execute ('''
    SELECT STRFTIME('%Y', timestamp) as year, COUNT(*)
    FROM (SELECT timestamp FROM nodes
    UNION SELECT timestamp FROM ways)
    GROUP BY year
    ORDER BY year DESC
    ''')
all_rows = c.fetchall()
pprint(all_rows)
db.close()

[(u'2016', 52011),
 (u'2015', 57476),
 (u'2014', 52523),
 (u'2013', 52881),
 (u'2012', 144792),
 (u'2011', 99937),
 (u'2010', 241083),
 (u'2009', 114439),
 (u'2008', 17638),
 (u'2007', 4653),
 (u'2006', 372)]


In [17]:
db = sqlite3.connect(sqlite_file)
c = db.cursor()
# Which months of which years had the most ways and nodes added to the database
c.execute ('''
    SELECT STRFTIME('%Y-%m', timestamp) as yearMonth, COUNT(*)
    FROM (SELECT timestamp FROM nodes
    UNION SELECT timestamp FROM ways)
    GROUP BY yearMonth
    ORDER BY COUNT(*) DESC
    LIMIT 5
    ''')
all_rows = c.fetchall()
pprint(all_rows)
db.close()

[(u'2010-09', 75669),
 (u'2010-04', 59329),
 (u'2010-07', 58310),
 (u'2012-04', 33734),
 (u'2012-05', 29342)]


In [18]:
db = sqlite3.connect(sqlite_file)
c = db.cursor()
# In the year-month with the highest way/node entries, which users were adding to the database and how many
# entries did each user contribute
c.execute ('''
    SELECT user, COUNT(*)
    FROM (SELECT timestamp, user FROM nodes
    UNION SELECT timestamp, user FROM ways),
    (SELECT STRFTIME('%Y-%m', timestamp) as yearMonth
    FROM (SELECT timestamp FROM nodes
    UNION SELECT timestamp FROM ways)
    GROUP BY yearMonth
    ORDER BY COUNT(*) DESC
    LIMIT 1) as subq
    WHERE STRFTIME('%Y-%m', timestamp) = yearMonth
    GROUP BY user
    ORDER BY COUNT(*) DESC
    ''')
all_rows = c.fetchall()
pprint(all_rows)
db.close()

[(u'andrewpmk', 72535),
 (u'Victor Bielawski', 2949),
 (u'bdustan', 101),
 (u'mfagan', 25),
 (u'MikeyCarter', 23),
 (u'tixuwuoz', 18),
 (u'bgibbard', 13),
 (u'brandoncote', 10),
 (u'salocinbake', 8),
 (u'emvee', 7),
 (u'Andre68', 2),
 (u'Sven L', 1),
 (u'de239', 1)]


### Discussion & Ideas for Improvements

After having run multiple queries over the full size database it was apparent that more cleaning could have been done. By looking at some of the top values in the cities key, it was evident that there were duplicate values for the same meaning, such as 'City of Toronto' and 'Toronto' as we had seen before. A full look into each of the way and node keys would allow you to further clean the data but figuring out when to stop would be difficult and would depend on the end use or reason for querying. For example, one might be more thorough if they were creating a navigation system out of this data as to not give anyone the wrong directions or misleading information.

Additionally, I think it would be interesting to include more graphics (graphs and charts) regarding the time/date/year that ways and nodes are being added. On top of this, there could be a plethora of statistics regarding which users are adding data and what days or times of days they are doing this.

Furthermore, there were a handful of tags that were either omitted or not fixed in the earlier stages of cleaning. A script could be written in which it tells the user how many tags or elements have not been fixed and then on a one by one basis either have the user either manually correct any unclean data or cross check some of the map data with another application such as google maps.

Some issues with these improvements are that they could become incredibly time consuming and difficult to know when to stop since we are dealing with such a large set of data.

### <span style='color:#5F04B4;'>References </span>

The following websites were refered to during the completion of this project.
- stackoverflow.com
- https://en.wikipedia.org/wiki/Street_suffix